# Assas Data Handler

In [1]:
 
#user = 'ke4920'
#password = 'R.adio_!1234'
#server = 'os.lsdf.kit.edu'
#share = 'kit\scc\projects\ASSAS'
#archive = 'archive'

#self.client_config = smbclient.ClientConfig(username=user,password=password)
#self.session = smbclient.register_session(server, username=user, password=password)

#smbclient.mkdir(r"\\os.lsdf.kit.edu\kit\scc\projects\ASSAS\test", username="ke4920", password="R.adio_!1234")
#
#with smbclient.open_file(r"\\os.lsdf.kit.edu\kit\scc\projects\ASSAS\test\file.txt", mode="w+") as fd:
#    fd.write(u"file contents")

In [17]:
import os
import time
import uuid
import shutil
import h5py
import numpy as np

from pymongo import MongoClient

class DatabaseHandler:

    def __init__(self, connectionstring):
        
        self.client = MongoClient(connectionstring)

        self.db_handle = self.client["assas"]
        self.file_collection = self.db_handle["files"]

    def get_db_handle(self):

        return self.db_handle

    def get_file_collection(self):
        
        return self.file_collection
    
    def insert_file_document(self, file):

        self.file_collection.insert_one(file)

class AssasDataset:
    
    def __init__(self, mountpoint, archive_dir, tmp_dir):
        
        self.mount_point = mountpoint
        self.archive_dir = archive_dir
        self.tmp_dir = tmp_dir
        
        self.uuid = str(uuid.uuid4())
        self.upload_time = str(time.time())
        self.filename = "dataset.h5"
        self.path = self.mount_point + self.archive_dir + str(self.uuid) + "/"

    def read_binary(self):

        self.name = "scenario_x"
        self.variables = ["var1", "var2"]
        self.channels = 4
        self.meshes = 16
        self.samples = 1000

    def create_lsdf_archive(self):

        if not os.path.isdir(self.mount_point + self.archive_dir):
            os.mkdir(self.mount_point + self.archive_dir)
        
        if not os.path.isdir(self.path):
            os.mkdir(self.path)
    
    def create_hdf5(self):

        with h5py.File(self.path + self.filename, 'w') as h5f:

            h5f.create_group('metadata')
            h5f['metadata'].attrs['upload_time'] = 0

            h5f.create_group('input')
            h5f['input'].attrs['debris'] = 0

            data_group = h5f.create_group('data')

            for variable in self.variables:
                variable_group = data_group.create_group(variable)
                array = np.random.rand(self.channels, self.meshes, self.samples, 1).reshape(self.channels, self.meshes, self.samples)
                variable_group.create_dataset(variable, data = array)

        h5f.close()

    def get_file_document(self):

        return {"uuid": self.uuid, "name": self.name, "path": self.path, "upload_time": self.upload_time}

class DatabaseManager:

    def __init__(self):
        
        self.connectionstring = "mongodb://localhost:27017/"
        self.mountpoint = "/mnt/ASSAS/"
        self.archive_dir = "test/"
        self.tmp_dir = "./tmp"

        self.database_handler = DatabaseHandler(self.connectionstring)

    def upload(self):

        assas_dataset = AssasDataset(self.mountpoint, self.archive_dir, self.tmp_dir)
        
        assas_dataset.read_binary()

        assas_dataset.create_lsdf_archive()
        
        assas_dataset.create_hdf5()

        dataset_file_document = assas_dataset.get_file_document()
        print(dataset_file_document)

        self.database_handler.insert_file_document(dataset_file_document)
        
    def view(self):
        
        file_collection = self.database_handler.get_file_collection()
        print(file_collection)

        for file in file_collection.find():
            print(file)


In [18]:
database_manager = DatabaseManager()

In [22]:
database_manager.upload()

{'uuid': '3cc6a43e-ab55-4754-8eaf-ddca3c9563fc', 'name': 'scenario_x', 'path': '/mnt/ASSAS/test/3cc6a43e-ab55-4754-8eaf-ddca3c9563fc/', 'upload_time': '1697547252.5545473'}


In [23]:
database_manager.view()

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'assas'), 'files')
{'_id': ObjectId('652e7f7f266b9bc08ba9ecf3'), '1': 1, '2': 2}
{'_id': ObjectId('652e82ab7f534891b358fef3'), 'uuid': '1766e6d8-3ac4-4b02-833d-1dae8d175453', 'name': 'scenario_x', 'path': '/mnt/ASSAS/test/1766e6d8-3ac4-4b02-833d-1dae8d175453/', 'upload_time': '1697546922.6933124'}
{'_id': ObjectId('652e830f7f534891b358fef5'), 'uuid': '10716c2b-6db6-48cf-84a6-4e0c94d86329', 'name': 'scenario_x', 'path': '/mnt/ASSAS/test/10716c2b-6db6-48cf-84a6-4e0c94d86329/', 'upload_time': '1697547022.6108284'}
{'_id': ObjectId('652e83e77f534891b358fef7'), 'uuid': 'b245c3f3-fa3d-4cdd-ad94-2a793dd1f073', 'name': 'scenario_x', 'path': '/mnt/ASSAS/test/b245c3f3-fa3d-4cdd-ad94-2a793dd1f073/', 'upload_time': '1697547238.279937'}
{'_id': ObjectId('652e83f57f534891b358fef8'), 'uuid': '3cc6a43e-ab55-4754-8eaf-ddca3c9563fc', 'name': 'scenario_x', 'path': '/mnt/ASSAS/test/3cc6a43e-ab55-4